In [3]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os
import numpy
from keras.utils import np_utils

WEIGHTS_FILE = "S120-L256-L256-weights-improvements-059-2.5130.hdf5"

# VARS
DATA_GLOB = "data/pokemon-single/PkmRB/*.mid"
MODEL_DIR = "model/pokemon-single/PkmRB/S120-L256-L256/"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")
NOTES_INPUT_PARSED_FILE = "%s/%s" % (MODEL_DIR, "notes-input-parsed.json")
CHECKPOINT_FILE = "%s/%s" % (MODEL_DIR, WEIGHTS_FILE)
SEQUENCE_LENGTH = 120

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())
if NOTES_INPUT_PARSED_FILE:
    with open(NOTES_INPUT_PARSED_FILE, 'r') as notes_input_parsed_file:
        NOTES_BY_FILE = json.loads(notes_input_parsed_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))
N_VOCAB = len(NOTES)

print("Num Vocab: %d" % N_VOCAB)

Num Vocab: 50


In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    print("Loading Checkpoint File: %s" % CHECKPOINT_FILE)
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

NameError: name 'X' is not defined

In [44]:
import uuid
from music21 import note, chord, stream

MIDI_OUTPUT_DIRECTORY = "output/pokemon/PkmRB-V1/"

SEED_INPUT_SEQUENCE = X[numpy.random.randint(0, len(X))] * N_VOCAB
SEED_SEQUENCE = [INT_TO_NOTE[numpy.round(number[0])] for number in SEED_INPUT_SEQUENCE]

print('|'.join(SEED_SEQUENCE))

pattern = SEED_INPUT_SEQUENCE
output = []

for i in range(500):
    input_sequence = numpy.reshape(pattern, (1, len(pattern), 1))
    output_sequence = model.predict(input_sequence, verbose=0)
    
    number = numpy.argmax(output_sequence)
    result = INT_TO_NOTE[number]
    output.append(result)
    
    pattern = numpy.append(pattern, number)
    pattern = pattern[1:]
    
offset = 0
output_notes = []
for note_pattern in output:
    if ('.' in note_pattern) or note_pattern.isdigit():
        notes_in_chord = note_pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note = storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(note_pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

if not os.path.exists(os.path.dirname(MIDI_OUTPUT_DIRECTORY)):
    os.makedirs(os.path.dirname(MIDI_OUTPUT_DIRECTORY))
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp="%s/%s.%s.%s" % (MIDI_OUTPUT_DIRECTORY, os.path.basename(CHECKPOINT_FILE), uuid.uuid4(), "mid"))

E4|F#4|A4|E4|A4|E4|F#4|A4|G4|E4|E4|A4|A4|A4|C#5|E4|E4|A4|A4|A4|C#5|E4|C#5|F#5|A4|E4|B4|E5|A4|A4|G4|G4|C#5|A4|F#4|D5|F#4|A4|F#5|A4|B4|F#4|C#5|A4|D5|A4|F#4|C#5|A4|B4|E4|G4|A4|A4|F#4|D5|D4|F4|A4|F#4|F#4|D5|D4|D5|A5|F#4|D4|C#5|G5|F#4|B4|G4|A4|F#5|F#4|B4|E5|E4|A4|C#5|A4|G4|E4|F#4|A4|E4|A4|E4|F#4|A4|G4|E4|E4|A4|A4|A4|C#5|E4|E4|A4


'output/pokemon/PkmRB-V1//LSTM512-weights-improvement-27-0.8165.hdf5.bd8bbe50-3fdd-4e88-88ba-6e730d99f785.mid'